In [1]:
import random
import numpy as np
from collections import deque
import pandas as pd 

# Parámetros

In [2]:
#Horizonte de simulacion (Turnos)
Turnos = 1
Reloj = 0
tiempo_aculatamiento = 70/60

In [3]:
#Se leen las caracteristicas de las palas
dfPalas = pd.read_excel('Informacion Palas.xlsx',  1)
InfoPalas = list(zip(dfPalas[dfPalas.columns[0]].values, dfPalas[dfPalas.columns[1]].values, dfPalas[dfPalas.columns[2]].values, \
                            dfPalas[dfPalas.columns[3]].values, dfPalas[dfPalas.columns[4]].values, \
                            dfPalas[dfPalas.columns[5]].values, dfPalas[dfPalas.columns[6]].values, \
                            dfPalas[dfPalas.columns[7]].values, dfPalas[dfPalas.columns[8]].values))

#Lista de nodos de mineral y esteril
Nodos_Mineral = ["P1", "P2"]
Nodos_Esteril = ["P6", "P7"]
Polvorazos = Nodos_Mineral + Nodos_Esteril


#Número de camiones disponibles
total_camiones = 20

#Lista de stocks/botaderos
#ID, espacios, ubicacion, tiempo_atencion ID=ubicacion?
SoB = ["S1", "S2"]
espacios = 5
tiempo_atencion_stock = 70/60 #segundos

#Se lee la informacion del chancador
dfChancador = pd.read_excel('Informacion Palas.xlsx',  0)
InfoChancador = list(zip(dfChancador[dfChancador.columns[0]].values, dfChancador[dfChancador.columns[1]].values, dfChancador[dfChancador.columns[2]].values, \
                            dfChancador[dfChancador.columns[3]].values, dfChancador[dfChancador.columns[4]].values, \
                            dfChancador[dfChancador.columns[5]].values, dfChancador[dfChancador.columns[6]].values, \
                            dfChancador[dfChancador.columns[7]].values, dfChancador[dfChancador.columns[8]].values))

# Dinamico

In [4]:
def dinamico(destinos, palas):
    return (random.choice(palas), random.choice(destinos))

# 1. Clases
## 1.1. Palas

In [5]:
class Pala:
    def __init__(self, ID, capacidad, ubicacion, media_recuperacion, media_falla, velocidad, tiempo_aculatado, min_carga, moda_carga, max_carga):
        self.ubicacion = ubicacion #Nodo en que esta ubicada la pala
        self.capacidad = capacidad #Capacidad nominal de excavacion
        self.ID = ID #ID pala
        self.fila = deque() #camiones en fila para se atendidos
        
        self.espacios_aculatamiento = [0, 0] #camiones aculatados 
        self.orden_llegada = deque() #orden de llegada al aculatamiento
        
        self.horas_eventos = {"Recuperacion":99999999, "Caida":9999999, "Inicio Movimiento":99999999, "Fin Movimiento":99999999, "Carga":999999} 
        self.proximo_evento = None #Nombre del evento mas proximo
        self.toneladas_sacadas = {} #Diccionario de tuplas con toneladas sacadas por posicion/nodo y ley
        self.TPH = {} #Diccionario con los resultados del PL
        
        self.media_recuperacion = media_recuperacion
        self.media_falla = media_falla
        self.velocidad = velocidad
        
        self.tiempo_aculatado = tiempo_aculatado
        
        self.min = min_carga
        self.moda = moda_carga
        self.max = max_carga
        
        self.hora_siguiente = 999999999
        self.disponibilidad = True
                     
    def caida(self, Reloj):#Completar comn PL y con Dinamico
        self.estado = 0
        #POR FUERA:
        #Se debe correr el PL y luego el dinámico para todos los camiones en la fila, y en camion hacia la pala
        #for cada camion if camion hacia pala.ID correr dinamico 
        self.horas_eventos["Caida"] = 99999999
        self.horas_eventos["Recuperacion"] = Reloj + self.tiempo_recuperacion()
        if self.horas_eventos["Recuperacion"] > self.horas_eventos["Inicio Movimiento"]:
            self.horas_eventos["Inicio Movimiento"] = self.horas_eventos["Recuperacion"] + 0.0001 #Postergo el movimiento hasta que se arregle la pala
        self.proximo_evento = "Recuperacion"
        return None
    
    def recuperacion(self, Reloj):
        self.estado = 1
        self.horas_eventos["Recuperacion"] = 99999999
        self.horas_eventos["Caida"] = Reloj + self.tiempo_falla()
        self.proximo_evento = min(self.horas_eventos, key=self.horas_eventos.get)
        self.hora_siguiente = self.horas_eventos[self.proximo_evento]
        return None
    
    def disponible(self):
        for espacio in self.espacios_aculatamiento:
            if espacio != 0:
                return True
        return False
    
    def inicio_movimiento(self, Reloj, destino, MTV):
        pass
    
    def fin_movimiento(self, Reloj, Lista_movimientos_agendados):
        pass
       
    def tiempo_recuperacion(self):
        return(np.random.exponential(self.media_recuperacion))
    
    def tiempo_falla(self):
        return(np.random.exponential(self.media_falla))
    
    def hora_proximo_evento(self):
        self.hora_siguiente = min(self.horas_eventos.values())
        return(self.hora_siguiente)
    
    def nombre_proximo_evento(self):
        self.proximo_evento = min(self.horas_eventos, key=self.horas_eventos.get)
        return(self.proximo_evento)
    
    def duracion_atencion(self):
        return(np.random.triangular(self.min, self.moda, self.max))
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        

## 1.2. Camiones

In [6]:
class Camion:
    def __init__(self, ID):
        self.ID = ID
        self.Nombre = "C"+str(ID)
        self.destino = "CS03"
        self.origen = "CS03"
        self.almuerzo = 0 #0: No almorzo, 1 primera colacion, 2 segunda colacion (solo aplica turno B)
        self.hora_siguiente = 0
        self.descargas = {} #Diccionario que almacena la historia de las descargas: key = (Origen, Destino)| value = toneladas
        self.carga = 0
        #self.aculatandose = False
        self.estado = "Descargando" #Viajando, Aculatandose, Cargando, Descargando, Almorzando, En fila
        self.OD = None
        
    def descargar(self, nuevo_destino, Reloj, TdV):
        self.toneladas = 0
        self.origen = self.destino
        self.destino = nuevo_destino
        if self.carga > 0:
            self.descargas[(self.origen, destino)] = self.carga 
        self.carga = 0
        self.estado = "Viajando"
        self.hora_siguiente = Reloj + TdV
        return None
    
    def cargar(self, toneladas_cargadas, nuevo_destino):
        self.origen = self.destino
        self.destino = nuevo_destino
        self.carga = toneladas_cargadas
        return None
    
    def asignar_destino(self, Reloj, nuevo_destino, MTV):
        self.destino = nuevo_destino
        self.actualizar_estado("Viajando", Reloj + MTV[self.origen][self.destino])
    
    def actualizar_estado(self, Estado, Hora):
        self.estado = Estado 
        self.hora_siguiente = Hora
        
    def nuevo_viaje(self, Estado, Hora):
        self.estado = Estado 
        self.hora_siguiente = Hora
        self.origen = self.OD[0]
        self.destino = self.OD[1]
        
    def comenzar_almuerzo(self, destino, reloj, MTV, tiempo_colacion, almuerzo):
        self.origen = self.destino
        self.destino = destino
        self.estado = "Almorzando"
        self.hora_siguiente = Reloj + MTV[self.origen][self.destino] + tiempo_colacion
        self.almuerzo = almuerzo
        
    def __str__(self):
        return str(self.Nombre)
    def __unicode__(self):
        return str(self.Nombre)
    def __repr__(self):
        return str(self.Nombre)
        
        
        
        
        

## 1.3. Stock - Botadero

In [7]:
class Stock:
    
    def __init__(self, ID, espacios, ubicacion, tiempo_atencion, tiempo_aculatado):
        self.ID = ID
        self.fila = deque()
        self.espacios_aculatamiento = [0 for i in range(espacios)]
        self.ubicacion = ubicacion
        self.tiempo_atencion = tiempo_atencion
        self.hora_siguiente = 99999999
        self.tiempo_aculatado = tiempo_aculatado 
    
    def avanzar_fila(self):
        camion = self.fila.popleft()
        return camion
    
    def duracion_atencion(self):
        return(self.tiempo_atencion)
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
   

## 1.4. Chancador

In [8]:
class Chancador:
    
    def __init__(self, ID, ubicacion, media_falla, media_recuperacion, tiempo_aculatado, min_carga, moda_carga, max_carga):
        self.ID = ID
        self.ubicacion = ubicacion
        self.espacios_aculatamiento = [0, 0]
        self.fila = deque()
        self.media_falla = media_falla
        self.media_recuperacion = media_recuperacion
        self.tiempo_aculatado = tiempo_aculatado
        self.min = min_carga
        self.moda = moda_carga
        self.max = max_carga
        self.hora_siguiente = 999999999
        
    def tiempo_recuperacion(self):
        return(np.random.exponential(self.media_recuperacion))
    
    def tiempo_falla(self):
        return(np.random.exponential(self.media_falla))
    
    def hora_proximo_evento(self):
        return(min(horas_eventos.values()))
    
    def duracion_atencion(self):
        return(np.random.triangular(self.min, self.moda, self.max))
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
    

# 2. Simulación
## 2.1. Inicialización 

In [9]:
#Se evalúa que palas estarán operativas y en que condición y se generan las instancias 
L_Entidades = []
#Esteril: 303
#Mineral: 293
D_Palas = {}
D_Entidades = {}
for pala in InfoPalas:
    u = np.random.uniform(0,1)
    
    if 0 <= u and u < pala[3]:
        pass
    
    elif pala[3] <= u and u < 1:#pala[4]
        D_Palas[pala[0]] = Pala(pala[0], "TONELADAS X CARGA", random.choice(Polvorazos), pala[2], pala[1], "velocidad", \
                                pala[8], pala[5], pala[6], pala[7])
        L_Entidades.append(D_Palas[pala[0]])
        D_Entidades[pala[0]] = D_Palas[pala[0]]
       
    else:
        #D_Palas[pala[0]] = Pala(pala[0], "TONELADAS X CARGA", random.choice(Polvorazos), pala[2], pala[1], "velocidad", \
        #                        pala[8], pala[5], pala[6], pala[7]) 
        #D_Palas[pala[0]].recuperacion(0)#Se inicializa el tiempo de la siguiente falla
        #print(pala[0], D_Palas[pala[0]].hora_siguiente)
        #L_Entidades.append(D_Palas[pala[0]])
        #D_Entidades[pala[0]] = D_Palas[pala[0]]
        pass
        
#Se inicializan los camiones
D_Camiones = {}
for i in range(total_camiones):
    D_Camiones[i] = Camion(i)
    L_Entidades.append(D_Camiones[i])
    D_Entidades[i] = D_Camiones[i]


    
#Se inicializan los stocks/botaderos
D_Stocks = {}
for stock in SoB:
    D_Stocks[stock] = Stock(stock, espacios, random.choice(SoB), tiempo_atencion_stock, tiempo_aculatamiento)
    L_Entidades.append(D_Stocks[stock])
    D_Entidades[stock] = D_Stocks[stock]
#print("Stocks y Botaderos :", D_Stocks)    

#Se inicializa el Chancador
D_Chancador ={}
for chancador in InfoChancador:
    D_Chancador[chancador[0]] = Chancador(chancador[0], "CS03", chancador[1], chancador[2], chancador[8], \
                                          chancador[5], chancador[6], chancador[7])
    L_Entidades.append(D_Chancador[chancador[0]])
    D_Entidades[chancador[0]] = D_Chancador[chancador[0]]
    
#print("Chancador: ", D_Chancador)


L_Entidades.sort(key=lambda x: x.hora_siguiente, reverse=False)


## 2.2. Matriz de tiempos de viaje

In [10]:
MTV = {}
Destinos = []
Palas = []

for destino1 in L_Entidades:
    if destino1.__class__.__name__ in ["Stock", "Chancador"]:
        Destinos.append(destino1.ID)
    if destino1.__class__.__name__ == "Pala":
        Palas.append(destino1.ID)
    if destino1.__class__.__name__ in ["Stock", "Chancador", "Pala"]:
        MTV[destino1.ID] = {}
        for destino2 in L_Entidades:
            if destino2.__class__.__name__ in ["Stock", "Chancador", "Pala"]:
                MTV[destino1.ID][destino2.ID] = random.uniform(20, 40)

MTV["CASINO"] = {}
for key in MTV.keys():
    MTV[key]["CASINO"] = random.uniform(20, 40)
    MTV["CASINO"][key] = random.uniform(20, 40)


In [11]:
#Identificamos el siguiente evento que va a ocurrir}
TurnoA = True
TurnoB = False
NTurnos = 0

trucks = 0 
Entidad = L_Entidades[0]
Reloj = Entidad.hora_siguiente
while Reloj < 12*60:
    
    NTurnos = int(Reloj/(12*60))
    
    
    if int(Reloj/(12*60))%2 == 0:
        TurnoB = False
        TurnoA = True
    else:
        TurnoA = False
        TurnoB = True

    #Tomamos el primer elemento de la lista de Entidades
    Entidad = L_Entidades[0]
    Reloj = Entidad.hora_siguiente
    print('------------------------------------------------------------')
    print(Entidad, Reloj)
   
    #En caso de que la entidad sea un camion
    if Entidad.__class__.__name__ == "Camion":
        print(Entidad, Entidad.origen, Entidad.destino, Entidad.estado, Reloj, Entidad.almuerzo)
        if Entidad.estado == "Almorzando":

            Entidad.estado = "Descargando"
        
            print("El camion {} termino de almorzar.".format(Entidad))
            
        if Entidad.estado == "Descargando":#Salida de un stock, botadero o chancador
            #Comienzo de almuerzos
            
            #Almuerzo turno A
            MIT = NTurnos*60*12 #Minuto en que inicio turno
            if (TurnoA and 0 <= Entidad.ID and Entidad.ID <= 40 and Reloj >= (4.5*60 + MIT) and Entidad.almuerzo == 0):
                Destino = D_Entidades[Entidad.destino]
                print("El camion {} se va a almorzar desde {}".format(Entidad, Destino))
                print("La cola en el destino es {}".format(Destino.fila))
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo("CASINO", Reloj, MTV, 60, 1)
                print("La cola en el destino queda {}".format(Destino.fila))
                
                
            if (TurnoA and Entidad.ID > 40 and Reloj >= (6*60 + MIT) and Entidad.almuerzo ==0):
                Destino = D_Entidades[Entidad.destino]
                print("El camion {} se va a almorzar desde {}".format(Entidad, Destino))
                print("La cola en el destino es {}".format(Destino.fila))
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo("CASINO", Reloj, MTV, 60, 1)
                print("La cola en el destino queda {}".format(Destino.fila))
            
            #Primera colacion turno B
            if (TurnoB and 0 <= Entidad.ID and Entidad.ID <= 27 and Reloj >= (5*60 + MIT) and Entidad.almuerzo == 0):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 40, 1)
                
            if (TurnoB and 28 <= Entidad.ID and Entidad.ID <= 54 and Reloj >= (5.5*60 + MIT)and Entidad.almuerzo == 0):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 40, 1)
                
            if (TurnoB and 55 <= Entidad.ID and Reloj >= (4.5*60 + MIT) and Entidad.almuerzo == 0):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 40, 1)
                
            #Segunda colacion turno B
            if (TurnoB and 0 <= Entidad.ID and Entidad.ID <= 45 and Reloj >= (9*60 + MIT) and Entidad.almuerzo == 1):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 20, 2)
                
            if (TurnoB and 46 <= Entidad.ID and Reloj >= (9.5*60 + MIT) and Entidad.almuerzo == 1):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 20, 2)
            
            #En caso contrario, hay que asignarle un nuevo destino corriendo el dinámico
            if Entidad.estado == "Descargando": #Posible fuente de error; probar -- (if Entidad.estado == "Descargando")
                #Pregunto si hay fila en el stock/botadero/chancador
                try:
                    Destino = D_Entidades[Entidad.destino]
                    if len(Destino.fila) > 0:
                        #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                        i = Destino.espacios_aculatamiento.index(Entidad)

                        print("La fila en {} es {}".format(Destino, Destino.fila))
                        print("El lugar de aculatado en {} es {}".format(Destino, Destino.espacios_aculatamiento))
                        Siguiente = Destino.fila.popleft()
                        Destino.espacios_aculatamiento[i] = Siguiente
                        Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)

                        print("El camion {} sale y comienza el aculatado {}".format(Entidad, Siguiente))
                        print("La fila en {} quedo {}".format(Destino, Destino.fila))
                        print("El lugar de aculatado en {} quedo {}".format(Destino, Destino.espacios_aculatamiento))

                    elif Entidad in Destino.espacios_aculatamiento:
                        i = Destino.espacios_aculatamiento.index(Entidad)
                        print("No habia fila")
                        print("El lugar de aculatado en {} es {}".format(Destino, Destino.espacios_aculatamiento))
                        Destino.espacios_aculatamiento[i] = 0
                        print("El lugar de aculatado en {} quedo {}".format(Destino, Destino.espacios_aculatamiento))
                except:
                    pass

                nuevo_destino = dinamico(Destinos, Palas) #Decido hacia donde enviar el camion
                Entidad.OD = nuevo_destino #Guardo el resultado del dinamico - par OD
                print("El resultado del dinamico fue: {}".format(nuevo_destino))
                #Finalizo descarga, actualizo tiempo de sgte evento, y cambio estado a "Viajando", y origen
                Entidad.descargar(nuevo_destino[0], Reloj, MTV[Entidad.destino][nuevo_destino[0]]) 
        
        elif Entidad.estado == "Viajando":
            Destino = D_Entidades[Entidad.destino]
            #print("Entre")
            if len(Destino.fila) > 0:#Reviso si hay fila en el destino
                Destino.fila.append(Entidad)#El camion se pone a la fila
                Entidad.actualizar_estado("En fila", Reloj*99)
                print("Camion {} se pone a la fila {}".format(Entidad, Destino.fila))
            
            if len(Destino.fila) == 0:
                #Se revisa la disponibilidad de espacios de aculatamiento
                if 0 in Destino.espacios_aculatamiento:
                    #Camion comienza a aculatarse
                    try:
                        print("No hay fila, {} se aculata {}".format(Entidad, Destino.orden_llegada))
                    except:
                        pass
                    Entidad.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                    #El camion se ubica en el lugar disponible
                    lugar = 0
                    for espacio in Destino.espacios_aculatamiento:
                        if espacio == 0:
                            Destino.espacios_aculatamiento[lugar] = Entidad
                            if Destino.__class__.__name__ == "Pala":
                                Destino.orden_llegada.append(Entidad)
                                
                                print("Camion aculatandose en {}:{}".format(Destino, Destino.espacios_aculatamiento))
                            break
                        lugar += 1
                else:
                    Destino.fila.append(Entidad)
                    Entidad.actualizar_estado("En fila", Reloj*99)  
                    print("Esta es la fila ahora: {}".format(Destino.fila))
        
        elif Entidad.estado == "Aculatandose":#Termina de aculatarse
            Destino = D_Entidades[Entidad.destino]
            
            if Destino.__class__.__name__ in ["Chancador", "Stock"]:
                Entidad.actualizar_estado("Descargando", Reloj + Destino.duracion_atencion())#Comienza descarga solo
                #Destino.hora_siguiente = Reloj + Destino.duracion_atencion()
                #Entidad.actualizar_estado("Descargando", Reloj*99999)
                
            if Destino.__class__.__name__ == "Pala":
                Entidad.actualizar_estado("Cargando", Reloj*99)
                Primero = Destino.orden_llegada.popleft()
                #Reviso si hay alguien mas siendo atendido:
                if Destino.disponibilidad and Entidad == Primero:#Si no hay nadie
                    print("El primero en aculatarse: {}".format(Entidad))
                    Destino.horas_eventos["Carga"] = Reloj + Destino.duracion_atencion()
                    Destino.hora_proximo_evento()#Actualizamos la hora del evento siguiente
                    Destino.disponibilidad = False
                    print("Camion termino de aculatarse y comienza atencion. Fin atencion: {}".format(Destino.horas_eventos["Carga"]))
                else: #Si no hay disponibilidad, se espera a que termine la atencion en el lugar de aculatamiento
                    print("{} No era el primero: {} o la pala no estaba disponible: {}".format(Entidad, Primero, Destino.disponibilidad))
                    pass
                Destino.orden_llegada.insert(0, Primero)
                
        #elif Entidad.estado == "Cargando":
        #    Destino = D_Entidades[Entidad.destino]
        #    Entidad.cargar(Destino.capacidad, Entidad.OD[1])
        #    Entidad.actualizar_estado("Viajando", Reloj + MTV[Entidad.origen][Entidad.destino])
    
    if Entidad.__class__.__name__ == "Pala":
        
        print(Entidad, Entidad.horas_eventos, Entidad.nombre_proximo_evento())
        if Entidad.nombre_proximo_evento() == "Carga":#Finaliza la carga del camion en atencion
            #Se busca el camion que termino la atencion
            print("Orden de llegada: {}".format(Entidad.orden_llegada))
            Camion = Entidad.orden_llegada.popleft()
            #Despacho al camion:
            i = Entidad.espacios_aculatamiento.index(Camion)
            Entidad.espacios_aculatamiento[i] = 0 
            
            #El camion sale del lugar de aculatamiento
            #Camion.cargar(Entidad.capacidad, Camion.OD[1])
            #Camion.actualizar_estado("Viajando", Reloj + MTV[Camion.origen][Camion.destino])
            Camion.nuevo_viaje("Viajando", MTV[Camion.OD[0]][Camion.OD[1]]+Reloj)
            print("Camion despachado: {}, llega a las: {} a {}".format(Camion, Camion.hora_siguiente, Camion.destino))
            Entidad.disponibilidad = True
            Entidad.horas_eventos["Carga"] = Reloj*99
            Entidad.hora_proximo_evento()
            print("Estado del orden de llegada {}".format(Entidad.orden_llegada))
            #Reviso si quedan camiones por atender
            if len(Entidad.orden_llegada) > 0:
                Camion = Entidad.orden_llegada.pop()
                print(Camion, Camion.estado)
                if Camion.estado == "Cargando":
                    Entidad.disponibilidad = False
                    
                   
                    Entidad.horas_eventos["Carga"] = Reloj + Entidad.duracion_atencion()
                    print("Comienzo a atender a: {} y termino a las {}".format(Camion, Entidad.horas_eventos["Carga"]))
                    Entidad.hora_proximo_evento()
                    print("Proximo evento: {}".format(Entidad.hora_siguiente))
                Entidad.orden_llegada.append(Camion)
            
            print(Entidad.disponibilidad)
            if len(Entidad.fila) > 0:
                Camion = Entidad.fila.popleft()
                Entidad.orden_llegada.append(Camion)
                i = Entidad.espacios_aculatamiento.index(0)
                Entidad.espacios_aculatamiento[i] = Camion
                Camion.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                print("Lugar aculatado: {}".format(Entidad.espacios_aculatamiento))
                print("Fila: {}".format(Entidad.fila))     
    

    try:        
        print(Entidad, Entidad.hora_siguiente, Entidad.destino, Entidad.estado)  
        print('\n')
    except:
        print(Entidad, Entidad.hora_siguiente)
        print('\n')
    print('------------------------------------------------------------')              
    trucks += 1    
    L_Entidades.sort(key=lambda x: x.hora_siguiente, reverse=False)
   

        

------------------------------------------------------------
C0 0
C0 CS03 CS03 Descargando 0 0
El resultado del dinamico fue: ('PA12', 'S2')
C0 25.24968718906411 PA12 Viajando


------------------------------------------------------------
------------------------------------------------------------
C1 0
C1 CS03 CS03 Descargando 0 0
El resultado del dinamico fue: ('PA12', 'CS03')
C1 25.24968718906411 PA12 Viajando


------------------------------------------------------------
------------------------------------------------------------
C2 0
C2 CS03 CS03 Descargando 0 0
El resultado del dinamico fue: ('PA15', 'CS03')
C2 25.47190479717827 PA15 Viajando


------------------------------------------------------------
------------------------------------------------------------
C3 0
C3 CS03 CS03 Descargando 0 0
El resultado del dinamico fue: ('PA15', 'S2')
C3 25.47190479717827 PA15 Viajando


------------------------------------------------------------
----------------------------------------

C9 82.57193416499794
C9 PA8 S2 Descargando 82.57193416499794 0
No habia fila
El lugar de aculatado en S2 es [C9, C4, 0, 0, 0]
El lugar de aculatado en S2 quedo [0, C4, 0, 0, 0]
El resultado del dinamico fue: ('PA10', 'S1')
C9 115.86263575843005 PA10 Viajando


------------------------------------------------------------
------------------------------------------------------------
C4 83.0184355504034
C4 PA15 S2 Descargando 83.0184355504034 0
No habia fila
El lugar de aculatado en S2 es [0, C4, 0, 0, 0]
El lugar de aculatado en S2 quedo [0, 0, 0, 0, 0]
El resultado del dinamico fue: ('PA9', 'S1')
C4 118.38282583365957 PA9 Viajando


------------------------------------------------------------
------------------------------------------------------------
PA6 83.09274583359873
PA6 {'Recuperacion': 99999999, 'Caida': 9999999, 'Inicio Movimiento': 99999999, 'Fin Movimiento': 99999999, 'Carga': 83.09274583359873} Carga
Orden de llegada: deque([C0])
Camion despachado: C0, llega a las: 111.66904

PA6 16409.77746283301


------------------------------------------------------------
------------------------------------------------------------
PA21 166.21832268660657
PA21 {'Recuperacion': 99999999, 'Caida': 9999999, 'Inicio Movimiento': 99999999, 'Fin Movimiento': 99999999, 'Carga': 166.21832268660657} Carga
Orden de llegada: deque([C11])
Camion despachado: C11, llega a las: 198.79209359649911 a CS03
Estado del orden de llegada deque([])
True
PA21 16455.61394597405


------------------------------------------------------------
------------------------------------------------------------
C3 166.3055295655059
C3 PA9 S1 Aculatandose 166.3055295655059 0
C3 167.47219623217256 S1 Descargando


------------------------------------------------------------
------------------------------------------------------------
C7 166.92351555806553
C7 PA13 CS03 Descargando 166.92351555806553 0
No habia fila
El lugar de aculatado en CS03 es [C7, 0]
El lugar de aculatado en CS03 quedo [0, 0]
El resultad



------------------------------------------------------------
------------------------------------------------------------
C18 227.7976462973733
C18 CS03 PA6 Aculatandose 227.7976462973733 0
El primero en aculatarse: C18
Camion termino de aculatarse y comienza atencion. Fin atencion: 229.8687789879804
C18 22551.966983439957 PA6 Cargando


------------------------------------------------------------
------------------------------------------------------------
C2 228.3266765854102
C2 PA9 CS03 Viajando 228.3266765854102 0
C2 229.49334325207687 CS03 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C8 229.17803220018226
C8 PA9 CS03 Descargando 229.17803220018226 0
No habia fila
El lugar de aculatado en CS03 es [C8, C2]
El lugar de aculatado en CS03 quedo [0, C2]
El resultado del dinamico fue: ('PA21', 'CS03')
C8 251.70158504487523 PA21 Viajando


---------------------------------------------------------

C3 PA10 CS03 Viajando 300.63371326484594 0
C3 301.8003799315126 CS03 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C7 301.67357654275037
C7 PA12 S1 Descargando 301.67357654275037 0
El camion C7 se va a almorzar desde S1
La cola en el destino es deque([])
La cola en el destino queda deque([])
C7 401.0308287949551 CASINO Almorzando


------------------------------------------------------------
------------------------------------------------------------
C12 301.7329883510288
C12 S1 PA15 Aculatandose 301.7329883510288 0
El primero en aculatarse: C12
Camion termino de aculatarse y comienza atencion. Fin atencion: 306.070954953088
C12 29871.565846751848 PA15 Cargando


------------------------------------------------------------
------------------------------------------------------------
C3 301.8003799315126
C3 PA10 CS03 Aculatandose 301.8003799315126 0
C3 303.7755687817043 CS03 Descargando


-------

C15 PA13 S1 Viajando 450.34074728155645 1
C15 451.50741394822313 S1 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C4 450.3893261885011
C4 PA12 S1 Aculatandose 450.3893261885011 1
C4 451.5559928551678 S1 Descargando


------------------------------------------------------------
------------------------------------------------------------
C19 450.5581909917424
C19 PA8 S2 Viajando 450.5581909917424 1
C19 451.72485765840906 S2 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C15 451.50741394822313
C15 PA13 S1 Aculatandose 451.50741394822313 1
C15 452.6740806148898 S1 Descargando


------------------------------------------------------------
------------------------------------------------------------
C4 451.5559928551678
C4 PA12 S1 Descargando 451.5559928551678 1
No habia fila
El lugar de aculatado en S1 es [C4, C


------------------------------------------------------------
------------------------------------------------------------
C8 523.7264573871607
C8 CS03 PA20 Viajando 523.7264573871607 1
No hay fila, C8 se aculata deque([])
Camion aculatandose en PA20:[C8, 0]
C8 524.8931240538274 PA20 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C16 523.748350028719
C16 S2 PA10 Viajando 523.748350028719 1
No hay fila, C16 se aculata deque([C11])
Camion aculatandose en PA10:[C11, C16]
C16 524.9150166953856 PA10 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C11 524.0733827937712
C11 S1 PA10 Aculatandose 524.0733827937712 1
El primero en aculatarse: C11
Camion termino de aculatarse y comienza atencion. Fin atencion: 526.75595049792
C11 51883.26489658335 PA10 Cargando


----------------------------------------------------------

C13 594.751257746642 CS03 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C4 593.9318395246235
C4 PA21 S2 Aculatandose 593.9318395246235 1
C4 595.0985061912901 S2 Descargando


------------------------------------------------------------
------------------------------------------------------------
PA21 594.395132512156
PA21 {'Recuperacion': 99999999, 'Caida': 9999999, 'Inicio Movimiento': 99999999, 'Fin Movimiento': 99999999, 'Carga': 594.395132512156} Carga
Orden de llegada: deque([C11])
Camion despachado: C11, llega a las: 622.8999251644832 a S1
Estado del orden de llegada deque([])
True
PA21 58845.118118703445


------------------------------------------------------------
------------------------------------------------------------
C13 594.751257746642
C13 PA9 CS03 Aculatandose 594.751257746642 1
C13 596.5908186537081 CS03 Descargando


-----------------------------------------------------------

C3 CS03 PA21 Viajando 668.0913535674833 1
No hay fila, C3 se aculata deque([])
Camion aculatandose en PA21:[C3, 0]
C3 669.25802023415 PA21 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C3 669.25802023415
C3 CS03 PA21 Aculatandose 669.25802023415 1
El primero en aculatarse: C3
Camion termino de aculatarse y comienza atencion. Fin atencion: 670.108300467977
C3 66256.54400318085 PA21 Cargando


------------------------------------------------------------
------------------------------------------------------------
PA21 670.108300467977
PA21 {'Recuperacion': 99999999, 'Caida': 9999999, 'Inicio Movimiento': 99999999, 'Fin Movimiento': 99999999, 'Carga': 670.108300467977} Carga
Orden de llegada: deque([C3])
Camion despachado: C3, llega a las: 702.6820713778695 a CS03
Estado del orden de llegada deque([])
True
PA21 66340.72174632971


------------------------------------------------------------
--------

In [12]:
print(Entidad.destino)

CS03
